In [1]:
from ase import ase, io, neighborlist
from functools import partial
import numpy as np

In [2]:
def md(timestep, sim_length, data_file, temp, box):
    
    current_state = init(data_file, timestep)
    t = 0
    while t < sim_length:
        energy, forces = get_ef(current_state, timestep, box)
        new_positions = get_new_positions(current_state, timestep)
        new_velocities = get_new_velocities(current_state, timestep)
        t = t + timestep
        out = [current_state, sample(current_state), energies, forces]
    
    print("MD simulation finished")

### TODO: 
- write get_positions_from_file function
- write get_new_positions function
- write get_new_velocities function

In [3]:
#initialize the system with random kinetic energy
def init(data_file, temp):
    
    state = {}
    
    
    # probably need to extract more than positions - masses etc.
    data = get_data_from_file(data_file) #need to write this function, maybe return a sim-data class or something
    positions = data.positions
    n_at = len(positions)
    
    #initialize velocities
    #maxwell-boltzmann standard distribution (from TMP Chem)
    sigma = math.sqrt(2.0 * 1.98E-3 * self.temperature / 3)
    for atom in data:
        
    
    velocities = np.random.normal(0.0, sigma, 3)
    
    sumv = velocities.sum() / n_at
    sumv2 = sumv**2 / n_at
    fs = np.sqrt(3*temp/sumv2)
    velocities = (velocities - sumv)*fs
    prev_positions = positions - (velocities*timestep)
    
    state['positions'] = positions
    state['velocities'] = velocities
    state['prev_positions'] = prev_positions
    state['n'] = n_at
    
    return state

In [4]:
def get_radius_pbc(atom_1, atom_2, L):
    x = atom_1[0] - atom_2[0]
    y = atom_1[1] - atom_2[1]
    z = atom_1[2] - atom_2[2]
    x_min = x - L*round(x/L)
    y_min = y - L*round (y/L)
    z_min = z - L*round(z/L)
    return np.sqrt(x_min**2 + y_min**2 + z_min**2)

In [5]:
def get_ef(state, temp, box):
    energy = 0
    rc = box/2
    ecut =  4*(rc**-12 - rc**-6)
    n_at = state['n']
    forces = np.zeros(n_at)
    for i in range(n_at-1):
        for j in range(i+1,n_at):
            r = get_radius_pbc(state['positions'][i],state['positions'][j],box)
            if r < rc:
                r2i = 1 / (r**2)
                r6i = r2i ** 3 
                ff = (48 * r2i) * (r6i * (r6i - 0.5))
                f[i] += ff*r
                f[j] += -ff*r
                energy += 4*r6i*(r6i-1)-ecut
    
    return (energy, forces)

In [38]:
def verlet()

Docstring:
zeros(shape, dtype=float, order='C')

Return a new array of given shape and type, filled with zeros.

Parameters
----------
shape : int or tuple of ints
    Shape of the new array, e.g., ``(2, 3)`` or ``2``.
dtype : data-type, optional
    The desired data-type for the array, e.g., `numpy.int8`.  Default is
    `numpy.float64`.
order : {'C', 'F'}, optional, default: 'C'
    Whether to store multi-dimensional data in row-major
    (C-style) or column-major (Fortran-style) order in
    memory.

Returns
-------
out : ndarray
    Array of zeros with the given shape, dtype, and order.

See Also
--------
zeros_like : Return an array of zeros with shape and type of input.
empty : Return a new uninitialized array.
ones : Return a new array setting values to one.
full : Return a new array of given shape filled with value.

Examples
--------
>>> np.zeros(5)
array([ 0.,  0.,  0.,  0.,  0.])

>>> np.zeros((5,), dtype=int)
array([0, 0, 0, 0, 0])

>>> np.zeros((2, 1))
array([[ 0.],
      

In [41]:
xi = (3, 3, 3)
xj = (5, 5, 5)

rij = np.subtract(xi,xj)

In [42]:
rij

array([-2, -2, -2])